In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_teddynote import logging
logging.langsmith("Baekseung_openai_api", set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
Baekseung_openai_api


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [6]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt_template = PromptTemplate.from_template(template)

prompt1 = prompt_template.format(country="대한민국")
print(prompt1)
prompt2 = prompt_template.format(country="미국")
print(prompt2)

대한민국의 수도는 어디인가요?
미국의 수도는 어디인가요?


## Chain 생성

### LCEL(LangChain Expression Language)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

chain = prompt | model

In [4]:
input = {"topic": "인공지능 모델의 학습 원리", "how": "간단하게"}
response = chain.invoke(input)
print(response.content)

인공지능 모델의 학습 원리는 주로 데이터와 알고리즘을 기반으로 합니다. 기본적인 과정은 다음과 같습니다:

1. **데이터 수집**: 모델을 학습시키기 위해 필요한 데이터를 수집합니다. 이 데이터는 텍스트, 이미지, 음성 등 다양한 형태일 수 있습니다.

2. **데이터 전처리**: 수집한 데이터를 정제하고 변환하여 모델이 이해할 수 있는 형식으로 만듭니다. 이 과정에는 결측값 처리, 정규화, 범주형 변수 인코딩 등이 포함됩니다.

3. **모델 선택**: 문제에 적합한 알고리즘을 선택합니다. 예를 들어, 회귀 문제에는 선형 회귀 모델을, 분류 문제에는 결정 트리나 신경망을 사용할 수 있습니다.

4. **학습**: 선택한 모델에 데이터를 입력하여 학습을 진행합니다. 이 과정에서 모델은 입력 데이터와 정답(레이블) 간의 관계를 학습하게 됩니다. 일반적으로 손실 함수(loss function)를 최소화하는 방향으로 가중치를 조정합니다.

5. **검증**: 학습된 모델의 성능을 평가하기 위해 검증 데이터를 사용합니다. 이를 통해 모델이 새로운 데이터에 얼마나 잘 일반화되는지를 확인합니다.

6. **튜닝**: 모델의 성능을 개선하기 위해 하이퍼파라미터 조정, 추가 데이터 수집, 또는 다른 알고리즘을 시도하는 등의 작업을 수행합니다.

7. **배포**: 최종적으로 학습된 모델을 실제 환경에 배포하여 사용합니다.

이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있는 능력을 갖추게 됩니다.


### 출력파서(Output Parser)

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

output_parser = StrOutputParser()   # 출력을 문자열 형으로 출력하는 파서 (기본값은 AImessage)

chain = prompt | model | output_parser

input = {"topic": "인공지능 모델의 학습 원리", "how": "간단하게"}
response = chain.invoke(input)
print(response)

'인공지능 모델의 학습 원리는 주로 데이터와 알고리즘을 기반으로 합니다. 다음은 그 기본적인 과정입니다:\n\n1. **데이터 수집**: 모델을 학습시키기 위해 필요한 데이터를 수집합니다. 이 데이터는 이미지, 텍스트, 숫자 등 다양한 형태일 수 있습니다.\n\n2. **데이터 전처리**: 수집한 데이터를 정제하고 변환하여 모델이 이해할 수 있는 형태로 만듭니다. 예를 들어, 결측값 처리, 정규화, 범주형 변수 인코딩 등이 포함됩니다.\n\n3. **모델 선택**: 문제에 적합한 알고리즘(모델)을 선택합니다. 예를 들어, 회귀, 분류, 군집화 등의 문제에 따라 다양한 모델이 사용됩니다.\n\n4. **학습**: 선택한 모델에 데이터를 입력하여 학습을 진행합니다. 이 과정에서 모델은 입력 데이터와 정답(레이블) 간의 관계를 학습합니다. 주로 경량화된 손실 함수(loss function)를 최소화하는 방향으로 가중치를 조정합니다.\n\n5. **검증**: 학습된 모델의 성능을 평가하기 위해 검증 데이터를 사용합니다. 이 단계에서 모델이 얼마나 잘 일반화되는지를 확인합니다.\n\n6. **튜닝**: 모델의 성능을 개선하기 위해 하이퍼파라미터 조정, 추가 데이터 수집, 또는 다른 모델 시도 등을 통해 최적화를 진행합니다.\n\n7. **테스트**: 최종적으로 테스트 데이터를 사용하여 모델의 성능을 평가하고, 실제 환경에서의 적용 가능성을 검토합니다.\n\n이러한 과정을 통해 인공지능 모델은 주어진 문제를 해결할 수 있는 능력을 갖추게 됩니다.'

In [6]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  - Waiter: Hello! Welcome to our restaurant. How many people are in your party?
  - You: Hi! There are two of us.
  - Waiter: Great! Please follow me to your table. Here are your menus. Can I get you something to drink while you look?
  - You: Yes, I’d like a glass of water, please. And my friend will have a lemonade.
  - Waiter: Sure! I’ll be right back with your drinks. Have you decided on what to order?
  - You: Yes, I’ll have the grilled chicken salad, and my friend will have the spaghetti.
  - Waiter: Excellent choices! I’ll put that in for you. 
  - You: Thank you!

- 한글 해석:
  - 웨이터: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 몇 분이신가요?
  - 당신: 안녕하세요! 저희는 두 명입니다.
  - 웨이터: 좋습니다! 이쪽으로 오세요. 여기 메뉴입니다. 주문하시기 전 음료수 드릴까요?
  - 당신: 네, 물 한 잔 부탁드립니다. 제 친구는 레모네이드를 마실게요.
  - 웨이터: 알겠습니다! 음료수 바로 가져다 드리겠습니다. 주문은 하셨나요?
  - 당신: 네, 저는 그릴에 구운 치킨 샐러드로 할게요, 그리고 제 친구는 스파게티로 할게요.
  - 웨이터: 훌륭한 선택입니다! 주문 받아서 바로 드리겠습니다.
  - 당신: 감사합니다!


## LCEL 인터페이스

사용자 정의 체인을 가능한 쉽게 만들 수 있도록, [`Runnable`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. 

`Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다.

- [`stream`](#stream): 응답의 청크를 스트리밍합니다.
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다.
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다.

비동기 메소드도 있습니다.

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다.
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다.
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_teddynote import logging
logging.langsmith("Baekseung_openai_api", set_enable=True)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")

chain = prompt | model | StrOutputParser()

## stream: 실시간 출력

이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 

In [ ]:
for token in chain.stream({"topic": "멀티모달"}):
    print(token, end="", flush=True)

## invoke: 호출
`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [ ]:
chain.invoke({"topic": "ChatGPT"})

## batch: 배치(단위 실행)
함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다.

In [ ]:
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다

`config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

In [ ]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

## async stream: 비동기 스트림
함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [ ]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

## async invoke: 비동기 호출
`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.

In [ ]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
my_process = chain.ainvoke({"topic": "NVDA"})

In [ ]:
# 비동기로 처리되는 프로세스가 완료될 때까지 기다립니다.
await my_process

## async batch: 비동기 배치
함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다.

이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `topic` 에 대한 작업을 비동기적으로 처리하고 있습니다.

`await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.

In [ ]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [ ]:
await my_abatch_process

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때, 각 요소를 병렬로 실행합니다.

`langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다.

`ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `country`에 대한 **수도** 와 **면적** 을 구하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다.

이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `capital`와 `area`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.



In [ ]:
from langchain_core.runnables import RunnableParallel

chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | model
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | model
    | StrOutputParser()
)

combined = RunnableParallel(capital=chain1, area=chain2)
combined.invoke({"country": "대한민국"})

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.

`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.

In [ ]:
combined.batch([{"country": "대한민국"}, {"country": "미국"}])